In [7]:
from sklearn import preprocessing
from time import time
from sklearn.feature_extraction import DictVectorizer
import numpy as np
import csv
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_diabetes
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import silhouette_samples, silhouette_score
from operator import truediv
from sklearn.metrics import pairwise_distances
import pandas as pd
get_ipython().magic('matplotlib')

df=pd.read_csv('C:/diabetic_data_processed_withweight.csv',';')
df_grouped=df.groupby(['patient_nbr'], sort=True).last()
print("Shape grouped: ",df_grouped.shape)

to_del = ['encounter_id', 'patient_nbr','medical_specialty','payer_code']
print (to_del)
#Filter_selected cols
filtered_cols = [c for c in df_grouped.columns if (c not in to_del) ]#and ('ENF' not in c)
df_2 = df_grouped[filtered_cols]
print ("df_2",df_2.shape)
print(df_2.columns)

Using matplotlib backend: Qt5Agg
Shape grouped:  (71518, 49)
['encounter_id', 'patient_nbr', 'medical_specialty', 'payer_code']
df_2 (71518, 46)
Index(['race', 'gender', 'age', 'weight', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object

In [8]:
print(set(df_2['acarbose']))
#0 (No), 1 (Steady), 2 (Up), -2 (Down)

{0, 1, 2, -2}


In [9]:
medications=['metformin', 'metformin-pioglitazone','glimepiride','glipizide','repaglinide','nateglinide','pioglitazone','metformin-pioglitazone','glimepiride-pioglitazone','acarbose','miglitol','insulin']
groups=['biguanides','sulfonylureas','glinides','insulin','alpha glucosidase inhibitors','glitazones']
df_2['biguanides']=df_2['metformin']+df_2['metformin-pioglitazone']
df_2['sulfonylureas']=df_2['glimepiride']+df_2['glipizide']
df_2['glinides']=df_2['repaglinide']+df_2['nateglinide']
df_2['glitazones']=df_2['pioglitazone']+df_2['metformin-pioglitazone']+df_2['glimepiride-pioglitazone']
df_2['alpha glucosidase inhibitors']=df_2['acarbose']+df_2['miglitol']

C:\Users\laia.subirats\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\laia.subirats\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [15]:
monobiguanides=0
monosulfonylureas=0
monoglinides=0
monoinsulin=0
monoalpha=0
monoglitazones=0
numtherapies=0
biguasulfo=0
biguaglini=0
biguainsu=0
biguaalpha=0
biguaglita=0
sulfoglini=0
sulfoinsu=0
sulfoalpha=0
sulfoglita=0
gliniinsu=0
glinialpha=0
gliniglita=0
insualpha=0
insuglita=0
alphaglita=0
biguasulfoglini=0
biguasulfoinsu=0
biguasulfoalpha=0
biguasulfoglita=0
sulfogliniinsu=0
sulfoglinialpha=0
sulfogliniglita=0
gliniinsualpha=0
gliniinsuglita=0
insualphaglita=0
df_2=df_2.reset_index()
print(df_2.index)
for i in range(0,df_2.shape[0]):    
    numtherapies=0
    countbiguanides=0
    countsulfonylureas=0
    countglinides=0
    countinsulin=0
    countalpha=0
    countglitazones=0
    if df_2.loc[i,'biguanides']!=0:
        countbiguanides=1
        numtherapies=1
    if df_2.loc[i,'sulfonylureas']!=0:
        countsulfonylureas=1
        numtherapies=numtherapies+1
    if df_2.loc[i,'glinides']!=0:
        countglinides=1
        numtherapies=numtherapies+1
    if df_2.loc[i,'insulin']!=0:
        countinsulin=1
        numtherapies=numtherapies+1
    if df_2.loc[i,'alpha glucosidase inhibitors']!=0:
        countalpha=1
        numtherapies=numtherapies+1
    if df_2.loc[i,'glitazones']!=0:
        countglitazones=1
        numtherapies=numtherapies+1
    if (numtherapies==1):
        monobiguanides=monobiguanides+countbiguanides
        monosulfonylureas=monosulfonylureas+countsulfonylureas
        monoglinides=monoglinides+countglinides
        monoinsulin=monoinsulin+countinsulin
        monoalpha=monoalpha+countalpha
        monoglitazones=monoglitazones+countglitazones
    else:
        if(numtherapies==2):
            if (countbiguanides+countsulfonylureas==2):
                biguasulfo=biguasulfo+1
            else:
                if (countbiguanides+countglinides==2):
                    biguaglini=biguaglini+1
                else:
                    if (countbiguanides+countinsulin==2):
                        biguainsu=biguainsu+1
                    else:
                        if (countbiguanides+countalpha==2):
                            biguaalpha=biguaalpha+1
                        else:
                            if (countbiguanides+countglitazones==2):
                                biguaglita=biguaglita+1
                            else:
                                if (countsulfonylureas+countglinides==2):
                                    sulfoglini=sulfoglini+1
                                else:
                                    if (countsulfonylureas+countinsulin==2):
                                        sulfoinsu=sulfoinsu+1
                                    else:
                                        if (countsulfonylureas+countalpha==2):
                                            sulfoalpha=sulfoalpha+1
                                        else:
                                            if (countsulfonylureas+countglitazones==2):
                                                sulfoglita=sulfoglita+1
                                            else:
                                                if (countglinides+countinsulin==2):
                                                    gliniinsu=gliniinsu+1
                                                else:
                                                    if (countglinides+countalpha==2):
                                                        glinialpha=glinialpha+1
                                                    else:
                                                        if (countglinides+countglitazones==2):
                                                            gliniglita=gliniglita+1
                                                        else:
                                                            if (countinsulin+countalpha==2):
                                                                insualpha=insualpha+1
                                                            else:
                                                                if (countinsulin+countglitazones==2):
                                                                    insuglita=insuglita+1
                                                                else:
                                                                    if (countalpha+countglitazones==2):
                                                                        alphaglita=alphaglita+1
        else:
            if(numtherapies==3):
                if (countbiguanides+countsulfonylureas+countglinides==3):
                    biguasulfoglini=biguasulfoglini+1
                else:
                    if (countbiguanides+countsulfonylureas+countinsulin==3):
                        biguasulfoinsu=biguasulfoinsu+1
                    else:
                        if (countbiguanides+countsulfonylureas+countalpha==3):
                            biguasulfoalpha=biguasulfoalpha+1
                        else:
                            if (countbiguanides+countsulfonylureas+countglitazones==3):
                                biguasulfoglita=biguasulfoglita+1
                            else:
                                if (countsulfonylureas+countglinides+countinsulin==3):
                                    sulfogliniinsu=sulfogliniinsu+1
                                else:
                                    if (countsulfonylureas+countglinides+countalpha==3):
                                        sulfoglinialpha=sulfoglinialpha+1
                                    else:
                                        if (countsulfonylureas+countglinides+countglitazones==3):
                                            sulfogliniglita=sulfogliniglita+1
                                        else:
                                            if (countglinides+countinsulin+countalpha==3):
                                                gliniinsualpha=gliniinsualpha+1
                                            else:
                                                if (countglinides+countinsulin+countglitazones==3):
                                                    gliniinsuglita=gliniinsuglita+1
                                                else:
                                                    if (countinsulin+countalpha+countglitazones==3):
                                                        insualphaglita=insualphaglita+1
    

#Monotherapy
print("Biguanides: ", monobiguanides, monobiguanides/df_2.shape[0])
print("sulfonylureas: ",monosulfonylureas, monosulfonylureas/df_2.shape[0])
print("glinides: ",monoglinides, monoglinides/df_2.shape[0])
print("insulin: ", monoinsulin, monoinsulin/df_2.shape[0])
print("alpha: ",monoalpha, monoalpha/df_2.shape[0])
print("glitazones :",monoglitazones, monoglitazones/df_2.shape[0])

#Doubletherapy
print("Biguasulfo: ", biguasulfo, biguasulfo/df_2.shape[0])
print("Biguaglini: ", biguaglini, biguaglini/df_2.shape[0])
print("Biguainsu: ", biguainsu, biguainsu/df_2.shape[0])
print("Biguaalpha: ", biguaalpha, biguaalpha/df_2.shape[0])
print("Biguaglita: ", biguaglita, biguaglita/df_2.shape[0])
print("Sulfoglini: ", sulfoglini, sulfoglini/df_2.shape[0])
print("Sulfoinsu: ", sulfoinsu, sulfoinsu/df_2.shape[0])
print("Sulfoalpha: ", sulfoalpha, sulfoalpha/df_2.shape[0])
print("Sulfoglita: ", sulfoglita, sulfoglita/df_2.shape[0])
print("Gliniinsu: ", gliniinsu, gliniinsu/df_2.shape[0])
print("Glinialpha: ", glinialpha, glinialpha/df_2.shape[0])
print("Gliniglita: ", gliniglita, gliniglita/df_2.shape[0])
print("Insualpha: ", insualpha, insualpha/df_2.shape[0])
print("Insuglita: ", insuglita, insuglita/df_2.shape[0])
print("Alphaglita: ", alphaglita, alphaglita/df_2.shape[0])

#Tripletherapy
print("biguasulfoglini: ", biguasulfoglini, biguasulfoglini/df_2.shape[0])
print("biguasulfoinsu: ", biguasulfoinsu, biguasulfoinsu/df_2.shape[0])
print("biguasulfoalpha: ", biguasulfoalpha, biguasulfoalpha/df_2.shape[0])
print("biguasulfoglita: ", biguasulfoglita, biguasulfoglita/df_2.shape[0])
print("sulfogliniinsu: ", sulfogliniinsu, sulfogliniinsu/df_2.shape[0])
print("sulfoglinialpha: ", sulfoglinialpha, sulfoglinialpha/df_2.shape[0])
print("sulfogliniglita: ", sulfogliniglita, sulfogliniglita/df_2.shape[0])
print("gliniinsualpha: ", gliniinsualpha, gliniinsualpha/df_2.shape[0])
print("gliniinsuglita: ",gliniinsuglita, gliniinsuglita/df_2.shape[0])
print("insualphaglita: ", insualphaglita, insualphaglita/df_2.shape[0])

RangeIndex(start=0, stop=71518, step=1)
Biguanides:  5107 0.07140859643726055
sulfonylureas:  3979 0.055636343298190664
glinides:  404 0.005648927542716519
insulin:  23651 0.3306999636455158
alpha:  40 0.0005592997567046058
glitazones : 1175 0.016429430353197796
Biguasulfo:  1429 0.019980983808272044
Biguaglini:  123 0.001719846751866663
Biguainsu:  4574 0.06395592717917167
Biguaalpha:  17 0.00023770239659945747
Biguaglita:  462 0.006459912189938197
Sulfoglini:  40 0.0005592997567046058
Sulfoinsu:  3783 0.052895774490338096
Sulfoalpha:  26 0.0003635448418579938
Sulfoglita:  462 0.006459912189938197
Gliniinsu:  461 0.006445929696020582
Glinialpha:  6 8.389496350569087e-05
Gliniglita:  54 0.0007550546715512179
Insualpha:  31 0.0004334573114460695
Insuglita:  1436 0.02007886126569535
Alphaglita:  3 4.194748175284544e-05
biguasulfoglini:  7 9.787745742330602e-05
biguasulfoinsu:  1602 0.022399955256019465
biguasulfoalpha:  6 8.389496350569087e-05
biguasulfoglita:  205 0.0028664112531111047
